In [1]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="quick-starts-ws-148187")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="Project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-149665
Azure region: southcentralus
Subscription id: f9d5a085-54dc-4215-9ba6-dad5d86e60a0
Resource group: aml-quickstarts-149665


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cpu_cluster_name = "ML-Cluster"
compute_config = AmlCompute.provisioning_configuration("Standard_D2_V2", min_nodes=0, max_nodes=2)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
cpu_cluster.wait_for_completion(show_output=True)

Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.1, 1),
    "--max_iter": choice(50,100,200)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("./", compute_target=cpu_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps, 
                                    estimator=est, 
                                    policy=policy,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=16)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [18]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metric = best_run.get_metrics()

print(best_run)
print(best_run_metric)
best_run.get_file_names()
best_model=best_run.register_model(model_name="best_hdr_model", model_path=best_run.get_file_names()[-1], tags=best_run_metric)

# Save the best model
best_run.download_file(best_run.get_file_names()[-1], output_file_path='./outputs/')

Run(Experiment: Project1,
Id: HD_acfb43c6-6e52-42c3-97e7-74d242676415_4,
Type: azureml.scriptrun,
Status: Completed)
{'Regularization Strength:': 0.41552689765375406, 'Max iterations:': 200, 'Accuracy': 0.9122359796067007}


In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

datastore_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=datastore_path)

In [7]:
from train import clean_data
from sklearn.model_selection import train_test_split
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y)

AMLdata = x_train.join(y_train)
AMLdata.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
23494,45,1,0,0,0,11,5,18,5,999,...,0,0,0,0,0,0,0,1,0,0
21701,25,0,0,1,1,6,2,660,2,6,...,0,0,0,0,1,0,0,0,0,1
4482,29,1,0,0,0,6,1,155,1,999,...,0,0,0,0,1,0,0,0,0,1
15614,33,0,0,0,0,5,2,140,1,999,...,0,0,0,0,0,0,0,1,0,0
30822,51,0,0,0,0,5,3,64,5,999,...,0,0,0,1,0,0,0,0,0,0


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=AMLdata,
    label_column_name='y',
    n_cross_validations=3)

In [9]:
# Submit your automl run

ws = Workspace.from_config()
experiment = Experiment(ws, "Project1-AML")
run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Project1-AML,AutoML_c4dce31e-01ef-41b2-ae28-f45f9753398a,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [24]:
# Retrieve and save your best automl model.

best_run, best_model = run.get_output()
print(best_run.get_metrics(name="accuracy"))
print(run.summary())
print(best_model.steps[-1])

{'accuracy': 0.9177323999937946}
[['StackEnsemble', 1, 0.9155875914402737], ['VotingEnsemble', 1, 0.9177323999937946], ['XGBoostClassifier', 18, 0.9158708906569558], ['LightGBM', 17, 0.9154662371400376], ['GradientBoosting', 2, 0.9032453325040528], ['ExtremeRandomTrees', 2, 0.8888393529123838], ['LogisticRegression', 1, 0.9097604923746379], ['RandomForest', 5, 0.9014243499230812]]
('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(
    estimators=[('33', Pipeline(memory=None,
             steps=[('sparsenormalizer', Normalizer(copy=True, norm='max')),
                    ('xgboostclassifier',
                     XGBoostClassifier(booster='gbtree', colsample_bytree=0.9, eta=0.01, max_depth=4, max_leaves=0, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(
        dataset_samples=24712,
        dataset_features=120,
        dataset_classes=2,
        dataset_num_categori...
        iteration_timeout_param=None,
        feature_column_names=None